In [ ]:
!pip install faiss-cpu
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer
from underthesea import word_tokenize
import re
import pandas as pd

# Đường dẫn đến PhoBERT đã fine-tuned
model_path = "./model/phobert-medical-final"

# Chọn thiết bị chạy model (GPU nếu có)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer và model PhoBERT
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path).to(device)

def preprocess_text(text):
    """
    Tiền xử lý văn bản:
      - Xóa khoảng trắng thừa, xuống dòng
      - Chuyển về lowercase
      - Tách từ bằng underthesea
      - Xử lý trường hợp NaN hoặc rỗng
    """
    if pd.isna(text) or not str(text).strip():
        return ''

    # Làm sạch text
    text = str(text).lower().replace('\n', ' ').replace('\r', '')
    text = re.sub(' +', ' ', text).strip()

    # Tách từ bằng underthesea
    segmented_text = word_tokenize(text, format='text')

    # Nếu kết quả rỗng → trả về chuỗi rỗng
    if not segmented_text.strip():
        return ''

    return segmented_text

def get_embedding(text):
    """
    Tính embedding cho văn bản bằng kỹ thuật mean-pooling:
      - Tokenize
      - Lấy last_hidden_state
      - Mean pooling dựa trên attention mask
    Trả về: numpy vector (768 chiều đối với PhoBERT-base)
    """

    # Tiền xử lý + tách từ
    segmented_text = preprocess_text(text)

    # Tokenize thành tensor
    inputs = tokenizer(
        segmented_text,
        return_tensors="pt",
        truncation=True,
        max_length=256,
        padding="max_length"
    )

    # Đưa tensor sang GPU nếu có
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Disable gradient cho tốc độ nhanh hơn
    with torch.no_grad():
        outputs = model(**inputs)

    last_hidden_state = outputs.last_hidden_state          # (batch, seq, 768)
    attention_mask = inputs["attention_mask"]              # (batch, seq)

    # Expand mask để match kích thước hidden state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()

    # Tính tổng embedding có mask
    sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, dim=1)

    # Mẫu số (tổng số token hợp lệ)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Mean pooling
    embedding = sum_embeddings / sum_mask

    # Trả về numpy vector
    return embedding.cpu().numpy()[0]


Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/Luyen_Code/model/phobert-medical-final and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import pandas as pd
import faiss
import pickle
import numpy as np 

try:
    df = pd.read_excel("./data/DataBase.xlsx")
except:
    print("Lỗi: Không tìm thấy file")
    raise SystemExit()

print("Đang tạo embedding...")

corpus_embeddings = []   # danh sách vector embedding
metadata = []            # danh sách metadata tương ứng

for index, row in df.iterrows():

    content = f'{row["Tên bệnh"]} có triệu chứng là {row["Triệu chứng"]}'

    # Tính embedding từ PhoBERT
    vector = get_embedding(content)

    # Lưu vector vào danh sách
    corpus_embeddings.append(vector)

    # Metadata tương ứng cho từng vector
    metadata.append({
        "id": index,
        "ten_benh": row["Tên bệnh"],
        "trieu_chung": row["Triệu chứng"]
    })

embedding_dim = 768   # PhoBERT-base = 768 chiều

embeddings_np = np.array(corpus_embeddings).astype("float32")

index = faiss.IndexFlatL2(embedding_dim)   # khởi tạo faiss và dùng L2 distance
index.add(embeddings_np)                   # thêm vector vào index

print(f"Đã lưu {index.ntotal} vector vào FAISS index.")

# LƯU FILE INDEX FAISS
faiss.write_index(index, "./model/medical_symptoms.index")
print("Đã lưu file index → ./model/medical_symptoms.index")

# LƯU METADATA (pickle)
with open("./model/medical_symptoms.pkl", "wb") as f:
    pickle.dump(metadata, f)

print("Đã lưu metadata → ./model/medical_symptoms.pkl")


Đang tạo embedding....
Đã lưu 583 vector vào Faiss
Đã lưu file index
Đã lưu metadata


In [ ]:
def search_disease(query, top_k = 3):
  query_vector = get_embedding(query)
  query_vector = np.array([query_vector]).astype("float32")

  D, I = index.search(query_vector, top_k)

  print (f"Câu hỏi: {query}\n")
  print ("---- Kết quả ----")
  for i, idx in enumerate(I[0]):
    result = metadata[idx]
    distance = D[0][i]

    print (f"Top {i+1} (Khoản cách: {distance:.4f}:)")
    print (f"Bệnh: {result["ten_benh"]}")
    print (f"Triệu chứng: {result["trieu_chung"][:100]}...")

user_query = "Tôi bị đau bụng vùng trên rốn và hay bị ợ chua"
search_disease(user_query)

print ("---------------------------------------")

user_query_2 = "Sốt cao, nổi mẩn đỏ trên da"
search_disease(user_query_2)

Câu hỏi: Tôi bị đau bụng vùng trên rốn và hay bị ợ chua

---- Kết quả ----
Top 1 (Khoản cách: 14.3922:)
Bệnh: Đau thượng vị
Triệu chứng: Tùy vào nguyên nhân gây bệnh mà triệu chứng đau thượng vị khác nhau và sẽ có những triệu chứng đi kè...
Top 2 (Khoản cách: 14.5912:)
Bệnh: Viêm thực quản trào ngược
Triệu chứng: Triệu chứng viêm thực quản trào ngược bao gồm: tại thực quản và ngoài thực quản. Tại thực quảm điển ...
Top 3 (Khoản cách: 14.8990:)
Bệnh: Đau dạ dày
Triệu chứng: Đau dạ dày có thể xuất hiện tại vùng thượng vị ở chính giữa bụng, cũng có thể lệch sang bên trái hoặ...
---------------------------------------
Câu hỏi: Sốt cao, nổi mẩn đỏ trên da

---- Kết quả ----
Top 1 (Khoản cách: 17.7181:)
Bệnh: Nhiễm liên cầu khuẩn
Triệu chứng: Bệnh viêm họng do liên cầu khuẩn: đau họng, đau đầu, đau bụng, sốt cao từ 39 độ kèm theo nổi hạch vù...
Top 2 (Khoản cách: 18.3823:)
Bệnh: Viêm da cơ địa
Triệu chứng: Bệnh viêm da cơ địa có triệu chứng điển hình là da viêm đỏ, tróc vảy, chảy dịch, dày s